In [1]:
%display latex
from pseries_basis import *
from ore_algebra import *
n = PSBasis.n(PSBasis)

In [2]:
B = BinomialBasis(); B

Binomial basis (x) choose n

In [3]:
OE.<E> = OreAlgebra(QQ[x], ('E', lambda p : p(x=x+1), lambda p : 0))
x = OE.base().gens()[0]

In [170]:
def apply_operator_to_seq(operator, seq):
    r'''
        Method to apply an operator to a sequence.
        
        This method will be similar to the usual __call__ method from ``ore_algebra``, but with sequences that
        are given as functions.
        
        INPUT:
        
        * ``operator``: and operator with 1 generator (the shift) over the polynomial ring `\mathbb{R}[x]`.
        * ``seq``: a sequence in functional format. This means that we can call it with integer values and we 
          obtain the values of the sequence at each point.
          
        OUTPUT:
        
        A sequence in function format.
    '''
    if len(operator.parent().gens()) > 1:
        raise TypeError()
    E = operator.parent().gens()[0]
    v = operator.parent().base().gens()[0]
    coefficients = operator.coefficients(sparse=False)
    return lambda i : sum(coefficients[j](**{str(v):i})*seq(i+j) for j in range(len(coefficients)))

def get_converted_init(seq, bas, size):
    r'''
        Method that computes new initial values after a conversion.
        
        Let `((P_{n,k})_n)_k` be a basis of the sequence space and ``seq`` the
        functional representation of a sequence `(a_n)_n`. If we express this
        sequence in the form:
        
        .. MATH::
            
            a_n = \sum_{k=0}^n P_{n,k} c_k,
            
        then the sequence `(c_k)_k` can be explicitly computed by solving a linear system.
        
        This method solves such system for a fixed amount of elements (gibven by ``size``)
    '''
    inhom = vector([seq(i) for i in range(size)])
    M = Matrix([[bas[i](j) for i in range(size)] for j in range(size)])
    return list(M.solve_right(inhom))
    
def solution(operator, init):
    d = operator.order()
    required = max(
                    -min([el[0] for el in cat_bin.polynomial().lc().roots() if el[0] in ZZ]),
                    -min([el[0] for el in cat_bin.polynomial().lc().roots() if el[0] in ZZ]), 
                    d)
    if len(init) < required:
        raise ValueError(f"More data ({required}) is needed")
    @lru_cache
    def __aux_sol(n):
        if n < 0:
            return 0
        elif n < operator.order():
            return init[n]
        else:
            coeffs = operator.polynomial().coefficients()
            lc = coeffs.pop()
            return -sum(__aux_sol(n-d+i)*coeffs[i](n-d) for i in range(operator.order()))/lc(n-d)
    return __aux_sol

def eval_ore_operator(operator, ring=None,**values):
    r'''
        Method to evaluate ore operators
        
        This method evaluate operators from ``ore_algebra`` as they are polynomials. This allows to change the name 
        of the generators to try a iterative approach.
    '''
    gens = [str(el) for el in operator.parent().gens()]
    outer_vals = {el : values.get(el, 0) for el in gens}
    inner_vals = {el : values[el] for el in values if (not el in outer_vals)}
    coefficients = [el(**inner_vals) for el in operator.polynomial().coefficients()]
    monomials = [prod(
        outer_vals[str(g)]**(m.degree(g)) for g in operator.polynomial().parent().gens()
    ) for m in operator.polynomial().monomials()]
    result = sum(coefficients[i]*monomials[i] for i in range(len(monomials)))
    if ring != None:
        return ring(result)
    return result

class latex_str:
    def __init__(self, data):
        self.__data = data
        self.__latex = None
        
    def __repr__(self):
        return self.__data
    def __str__(self):
        return self.__data
    def _latex_(self):
        if self.__latex is None:
            self.__latex = self.__data.replace("*", "").replace("+0", "")
            
        return self.__latex

def print_recurrence(operator):
    r'''
        Method to print nicely the recurrence equation induced by an operator.
    '''
    if len(operator.parent().gens()) > 1:
        raise TypeError()
    E = operator.parent().gens()[0]
    v = operator.parent().base().gens()[0]
    coefficients = operator.coefficients(sparse=False)
    
    return latex_str(" + ".join(
        "(" + 
        str(coefficients[j]).replace(str(v),'n') + 
        ")*"
        "a_{n+" + str(j) + "}" for j in range(len(coefficients))) + " = 0")

In [171]:
# Catalan sequence
cat = lambda n : catalan_number(n)
cat_op = (x+2)*E - (4*x + 2)

In [172]:
cat_bin = eval_ore_operator(B.remove_Sni((B.recurrence(cat_op))), OE, Sn = E, n = x, Sni = 1);
cat_bin_bin = eval_ore_operator(B.remove_Sni((B.recurrence(cat_op))), OE, Sn = E, n = x, Sni = 1);

In [173]:
print_recurrence(cat_bin_bin)

(n + 3)*a_{n+0} + (-2*n - 2)*a_{n+1} + (-3*n - 3)*a_{n+2} = 0

In [174]:
in_in_cat = solution(cat_bin_bin, [1,1])

In [175]:
cat2 = lambda n : sum(sum(in_in_cat(k2)*binomial(k1,k2) for k2 in range(k1+1))* binomial(n,k1) for k1 in range(n+1))

In [176]:
[cat2(i) for i in range(10)]

[1,
 3,
 25/3,
 202/9,
 535/9,
 4202/27,
 163519/405,
 252592/243,
 67834301/25515,
 1035012997/153090]

In [177]:
in_cat = solution(cat_bin,[1,0])

In [178]:
[in_cat(i) for i in range(10)]

[1, 0, 1, -2/3, 1, -1, 17/15, -6/5, 9/7, -19/14]

In [180]:
[cat(i) == sum(in_cat(k)*binomial(i,k) for k in range(i+1)) for i in range(10)]

[True, True, True, False, False, False, False, False, False, False]

$$a_0 = \binom{0}{0} c_0 = c_0$$
$$a_1 = \sum_{k=0}^1 \binom{1}{k} c_k = \binom{1}{0} c_0 + \binom{1}{1} c_1 = c_0 + c_1$$

In [157]:
catalan_number(1)

1

In [169]:
B.remove_Sni((B.recurrence(cat_op)))

(n + 3)*Sn^2 + (-2*n - 2)*Sn - 3*n - 3

In [181]:
cat_bin

(-3*x - 3)*E^2 + (-2*x - 2)*E + x + 3

In [182]:
v = vector([1,2,3])
M = Matrix([[1,2,3],[4,5,6],[7,8,9]])

In [185]:
M*M.solve_right(v) == v

True

$$f(x) \in \mathbb{K}[[x]],\qquad f(x) = \sum_n c_n P_n(x)$$

$$(f_n)_n \in \mathbb{K}^\mathbb{N}$$